In [1]:
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from IPython.display import SVG
import ipywidgets as widgets
from ipywidgets import interact
from pathlib import Path
import json

In [2]:
def draw_reaction(rxn: str, sub_img_size: tuple = (300, 200), use_smiles: bool = True):    
    rxn = Chem.rdChemReactions.ReactionFromSmarts(rxn, useSmiles=use_smiles)
    return Draw.ReactionToImage(rxn, useSVG=True, subImgSize=sub_img_size)

In [3]:
krs = Path('/home/stef/quest_data/bottle/data') / 'sprhea' / 'sprhea_240310_v3_mapped_no_subunits.json'
with open(krs, 'r') as f:
    krs = json.load(f)

In [4]:
decarb = {k: v for k,v  in krs.items() if v['min_rule'] == 'rule0024'}

In [ ]:
kopts = list(decarb.keys())

@interact
def show_decarbs(key = widgets.Dropdown(value=kopts[0], options=kopts)):    
    rxn = decarb[key]
    print(rxn['imt_rules'])
    print(rxn['rhea_ids'])
    display(SVG(draw_reaction(rxn['smarts'])))

interactive(children=(Dropdown(description='key', options=('14', '55', '121', '734', '1067', '1162', '1565', '…

In [ ]:
'''
mol to nx
get k hop ego graph for all rc atoms
take union of ego graphs
convert back to mol / smiles


Worth converting to networkx at all?
What does equal mean? Isomorphic?
How much flexibility to I want in determining equal?

'''

def mol_to_nx(mol):
    G = nx.Graph()

    for atom in mol.GetAtoms():
        G.add_node(atom.GetIdx(),
                   atomic_num=atom.GetAtomicNum(),
                   is_aromatic=atom.GetIsAromatic(),
                   atom_symbol=atom.GetSymbol())
        
    for bond in mol.GetBonds():
        G.add_edge(bond.GetBeginAtomIdx(),
                   bond.GetEndAtomIdx(),
                   bond_type=bond.GetBondType())
        
    return G